In [9]:
!pip install snowflake-connector-python

In [10]:
import snowflake.connector
from google.colab import userdata

# Replace the placeholders with your actual Snowflake credentials
conn = snowflake.connector.connect(
    user = userdata.get('SNOWFLAKE_USER'),
    password = userdata.get('SNOWFLAKE_PASSWORD'),
    account= userdata.get('SNOWFLAKE_ACCOUNT'),
    warehouse='COMPUTE_WH',
    database='BIGDATA_GITHUB',
    schema='RAW',
    role='ACCOUNTADMIN'
)


In [11]:
cur = conn.cursor()
url = 'azure://matthewleffler1.blob.core.windows.net/kaggle-datasets/clean_data/repo_list_data/'

try:
    cur.execute("BEGIN;")
    cur.execute(f"""
        CREATE OR REPLACE STAGE azure_parquet_stage
          URL = '{url}'
          CREDENTIALS = (
            AZURE_SAS_TOKEN = '{userdata.get('AZURE_SAS_TOKEN')}'
          )
          FILE_FORMAT = (TYPE = PARQUET);
        """)
    cur.execute("COMMIT;")
    print(f"Successfullt created stage.")
except Exception as e:
    cur.execute("ROLLBACK;")
    print(f"Error creating database object: {e}")
finally:
    cur.close()


Successfullt created stage.


In [12]:
cur = conn.cursor()
table_name = "REPO_LIST_TABLE"

try:
    cur.execute("BEGIN;")
    cur.execute(f"""
      CREATE OR REPLACE TABLE {table_name} (
          id BIGINT,
          repo_created_at TIMESTAMP_NTZ,
          repo_default_branch VARCHAR,
          repo_description VARCHAR,
          repo_fork BOOLEAN,
          repo_forks_count BIGINT,
          repo_full_name VARCHAR,
          repo_has_wiki BOOLEAN,
          repo_id BIGINT,
          repo_language VARCHAR,
          repo_license VARCHAR,
          repo_open_issues BIGINT,
          repo_owner_id BIGINT,
          repo_pushed_at TIMESTAMP_NTZ,
          repo_size BIGINT,
          repo_stargazers_count BIGINT,
          repo_updated_at TIMESTAMP_NTZ
      );
          """)
    cur.execute("COMMIT;")
    print("Table created successfully.")
except Exception as e:
    cur.execute("ROLLBACK;")
    print(f"Error creating database object: {e}")
finally:
    cur.close()

Table created successfully.


In [ ]:
cur = conn.cursor()

try:
    cur.execute("BEGIN;")
    cur.execute(f"""
      COPY INTO {table_name}
      FROM @azure_parquet_stage
      FILE_FORMAT = (TYPE = PARQUET)
      MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
      ON_ERROR = CONTINUE;
          """)
    cur.execute("COMMIT;")
    print(f"Data loaded into {table_name} successfully.")
except Exception as e:
    cur.execute("ROLLBACK;")
    print(f"Error loading data: {e}")
finally:
    cur.close()